In [4]:
# Importing necessary Libraries
from langgraph.graph import  StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
import os

print("Libraries imported successfully✅")


Libraries imported successfully✅


In [5]:
# Load environment variables
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found! Please set it in your .env file.")

print("✅ API key loaded successfully")

✅ API key loaded successfully


#### Let's Initialize the LLM


In [6]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-4o-mini


#### Let's define the systems prompt


In [9]:
system_message = SystemMessage(
    content=(
        "You are a helpful, polite customer support representative. "
        "You ask clarifying questions, remember past user details, "
        "and provide clear, friendly assistance."
    )
)

### Let's create the Agent Function

In [21]:
def customer_support_agent(state: MessagesState) :
    messages = [system_message] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [AIMessage(content=response.content)]}

#### Now let's connect it in a Stategraph

In [22]:
pattern = StateGraph(MessagesState)

pattern.add_node("support_agent", customer_support_agent)
pattern.set_entry_point("support_agent")

#### let's create a memory checkpointer

In [23]:
memory = MemorySaver()

app = pattern.compile(checkpointer=memory)

In [24]:
def interactive_chat():
    """
    Run an interactive chat session.
    Type 'exit' or 'quit' to stop.
    """
    print("\n" + "="*70)
    print("🤖 Interactive Chat Started")
    print("Type your message and press Enter. Type 'exit' to quit.")
    print("="*70 + "\n")
    
    thread_id = "interactive_session2"
    
    while True:
        user_input = input("\n👤 You: ").strip()
        
        if user_input.lower() in ["exit", "quit"]:
            print("\n👋 Goodbye!\n")
            break
        
        if not user_input:
            continue
        
        # Get response
        result = llm.invoke(
            {"messages": [HumanMessage(content=user_input)]},
            config={"configurable": {"thread_id": thread_id}}
        )
        
        # Print agent's response
        agent_message = result["messages"][-1]
        print(f"\n🤖 Agent: {agent_message.content}")

# Uncomment to run interactive chat:
interactive_chat()


🤖 Interactive Chat Started
Type your message and press Enter. Type 'exit' to quit.



ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.